# controll the robot

## import libraries


In [45]:
import cv2
import os
import random
import numpy as np
import tensorflow.lite as tflite

import matplotlib.pyplot as plt


We need the .tflite file to run the model on the robot. So we start by importing the model. It is in the same folder as the notebook.

In [46]:
tflite_model_filename = "mini_proj_model_v01_06.tflite"

In [47]:
interpreter = tflite.Interpreter(tflite_model_filename)
interpreter.allocate_tensors()

INFO: Applying 1 TensorFlow Lite delegate(s) lazily.


In [48]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
input_shape = input_details[0]['shape']

In [49]:
print("The model expects input shape: ", input_shape)
print("The output shape is: ", output_details[0]['shape'])

The model expects input shape:  [ 1 64 64  1]
The output shape is:  [1 2]


## car setup

#### Initialize the car and camera classes

Create the racecar class

In [7]:
from jetracer.nvidia_racecar import NvidiaRacecar
car = NvidiaRacecar()

Create the camera class.

In [8]:
from jetcam.csi_camera import CSICamera
camera = CSICamera(width=224, height=224, capture_fps=65)

setup the gains and bias for the drive function

In [49]:
STEERING_GAIN = 1
STEERING_BIAS = 0.2
THROTTLE_GAIN = 0

car.steering_gain = STEERING_GAIN
print(car.steering_gain)
car.throttle_gain = THROTTLE_GAIN
print(car.throttle_gain)
car.steering_offset = STEERING_BIAS

1.0
0.0


In [33]:
def preprocess(image):
    assert image.shape == (224,224,3)
    # crop the top 80 pixels
    image = image[80:,:,:]
    # check the input shape of the model and if the last dimension is 1, then convert to grayscale
    if input_shape[3] == 1:
        # convert to grayscale
        image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    # resize to the input shape of the model
    image = cv2.resize(image, (input_shape[2], input_shape[1]))
    # Scale the images to the range of [0, 1]
    image = np.expand_dims(image, axis=-1)
    image = image.astype('float32')
    image = image / 255.0
    return image

## driving the car (fully autonomous)

In [34]:
# wa make a low pass filter to smooth the steering and throttle values, we use a array of the last 5 values
prev_steering = [0.0] * 5
prev_throttle = [0.0] * 5

def low_pass_filter(value, prev_values):
    prev_values.append(value)
    prev_values.pop(0)
    return sum(prev_values) / len(prev_values)

In [51]:
try:
    # main loop
    while True:
        image = camera.read()
        image = preprocess(image)
        # add a batch dimension
        image = np.expand_dims(image, axis=0)
        # set the input tensor
        interpreter.set_tensor(input_details[0]['index'], image)
        # run the inference
        interpreter.invoke()
        # get the output tensor
        output_data = interpreter.get_tensor(output_details[0]['index'])
        # get the steering and throttle values
        steering = - output_data[0][0]
        throttle = - output_data[0][1]

        # apply low pass filter
        # steering = low_pass_filter(steering, prev_steering)
        # throttle = low_pass_filter(throttle, prev_throttle)

        # we plot the image and the steering and throttle values
        print("steering: ", steering, "throttle: ", throttle)
        # plot the image, the steering angle and throttle and the real steering angle and throttle
        # also draw a arrow with the steering angle and throttle
        # the arrow will be baced on the middle bottom of the image and will point up for throttle and left or right for steering angle
        # the length of the arrow will be the throttle and the angle of the arrow will be the steering angle
        # make arrow position relative to the image size
        arrow_pos_x = int(image.shape[2] / 2)
        arrow_pos_y = int(image.shape[1])
        # make arrow length relative to the throttle
        arrow_length = int(throttle * image.shape[1])
        # make arrow angle relative to the steering angle
        arrow_angle = int(steering * 180)
        # make arrow thickness relative to the image size
        arrow_thickness = int(image.shape[1] / 50)
        # make arrow color
        arrow_color = (0, 255, 0)
        # make arrow head length relative to the image size
        arrow_head_length = int(image.shape[1] / 10)
        # make arrow head thickness relative to the image size
        arrow_head_thickness = int(image.shape[1] / 50)
        # draw the arrow
        image = cv2.arrowedLine(image[0], (arrow_pos_x, arrow_pos_y), (arrow_pos_x + int(arrow_length * np.cos(np.radians(arrow_angle))), arrow_pos_y - int(arrow_length * np.sin(np.radians(arrow_angle)))), arrow_color, arrow_thickness, tipLength=arrow_head_length)
        # display the image
        plt.imshow(image[0])
        plt.show()

        # set the car controls
        car.steering = steering
        car.throttle = throttle

except:
    car.throttle = 0.0
    raise

KeyboardInterrupt: 

## driving the car (semi-autonomous - manual throttle)

In [47]:
import socket

# create a socket
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
port = 1234
sock.bind(('', port))

# the button routine must only run once evey time the button is pressed
# so we need to keep track of the last state of the button
last_button_state = False

In [48]:
try:
    # main loop
    while True:
        image = camera.read()
        image = preprocess(image)
        # add a batch dimension
        image = np.expand_dims(image, axis=0)
        # set the input tensor
        interpreter.set_tensor(input_details[0]['index'], image)
        # run the inference
        interpreter.invoke()
        # get the output tensor
        output_data = interpreter.get_tensor(output_details[0]['index'])
        # get the steering and throttle values
        steering = output_data[0][0]
        throttle = output_data[0][1]
        # apply low pass filter
        steering = low_pass_filter(steering, prev_steering)
        # set the car controls
        car.steering = steering
        # get the throttle value from the socket
        data, addr = sock.recvfrom(1024)
        data = data.decode().split(",")
        # set the throttle value
        car.throttle = float(data[1])

except:
    car.throttle = 0.0
    raise

KeyboardInterrupt: 

## E stop

In [ ]:
car.throttle = 0.0